In [1]:
print("Importa librerie", end="", flush=True)
from keras_tuner import HyperModel, RandomSearch, Hyperband, BayesianOptimization
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, RepeatVector, Dropout, TimeDistributed, Dense
print(".", end="", flush=True)
from tensorflow.keras.models import Sequential
print(".", end="", flush=True)
from tensorflow.keras.regularizers import l2
print(".", end="", flush=True)
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from sklearn.preprocessing import PowerTransformer
print(".", end="", flush=True)
from sklearn.base import clone
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)


Importa librerie............=


In [2]:
scaler = PowerTransformer()
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = 1000
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo

elenco_features = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "Volume",
    "MACDh",
    "PSAR",
    "PSARaf",
    "SUPERT", 
    "TRIX",
    "ATR",
    "DM_OSC",
    "ADX",
    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]
elenco_target = [
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features = {col: idx for idx, col in enumerate(elenco_features)}
col_targets = {col: idx for idx, col in enumerate(elenco_target)}
n_features = len(col_features)
n_targets = len(col_targets)

class MixedHyperModel(HyperModel):
    def __init__(self, n_timesteps, n_features, n_targets, giorni_previsione):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.n_targets = n_targets
        self.giorni_previsione = giorni_previsione

    def build(self, hp):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(hp.Int('lstm_units_1', 20, 300, step=20),
                       input_shape=(self.n_timesteps, self.n_features)))
        model.add(RepeatVector(self.giorni_previsione))

        # Aggiunta di layer LSTM intermedi
        for i in range(hp.Int('num_lstm_layers', 1, 4)):
            model.add(LSTM(hp.Int(f'lstm_units_{i+2}', 20, 300, step=20), return_sequences=True,
                           kernel_regularizer=l2(hp.Float('l2_rate', 0, 5, sampling='log'))))
            model.add(Dropout(hp.Float(f'lstm_dropout_{i+2}', 0, 0.5, step=0.1)))

        # Aggiunta di layer Dense
        for i in range(hp.Int('num_dense_layers', 1, 4)):
            model.add(TimeDistributed(Dense(hp.Int(f'dense_units_{i}', 20, 300, step=20), activation='relu')))
            model.add(Dropout(hp.Float(f'dense_dropout_{i}', 0, 0.5, step=0.1)))

        model.add(TimeDistributed(Dense(self.n_targets, activation='linear')))

        model.compile(optimizer="adam",
                      loss='mean_absolute_percentage_error')

        return model

input_shape = (n_timesteps, n_features)
output_shape = (giorni_previsione, n_targets)
hypermodel = MixedHyperModel(n_timesteps, n_features, n_targets, giorni_previsione)



In [3]:
print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]

print("Download dati ticker validazione")
ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
ticker_val.index = ticker_val.index.date
print("Calcolo indicatori ticker validazione")
ticker_val = fx.crea_indicatori(ticker_val)
ticker_val.dropna(axis=0, inplace=True)

idx_val, X_val, Y_val = fx.to_XY(ticker_val, elenco_features, elenco_target, n_timesteps, giorni_previsione, addestramento=True)
X_val = X_val.reshape((-1, n_timesteps * n_features))
Y_val = Y_val.reshape((-1, giorni_previsione * n_targets))
X_scaler_val = clone(scaler)
Y_scaler_val = clone(scaler)
X_val = X_scaler_val.fit_transform(X_val)
Y_val = Y_scaler_val.fit_transform(Y_val)
X_val = X_val.reshape((-1, n_timesteps, n_features))
Y_val = Y_val.reshape((-1, giorni_previsione, n_targets))

if os.path.exists("X.npy") and os.path.exists("Y.npy"):
    X = np.load('X.npy')
    Y = np.load('Y.npy')
else:
    X = np.zeros((0, n_timesteps, n_features))
    Y = np.zeros((0, giorni_previsione, n_targets))
    for i_ticker in range (n_simboli_addestramento):
        nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
        print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
        print("Download dati ticker")
        try:
            ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
            if ticker["Close"].iloc[-1] >= 1:
                ticker.index = ticker.index.date
                print("Calcolo indicatori ticker")
                ticker = fx.crea_indicatori(ticker)
                ticker.dropna(axis=0, inplace=True)

                print("Definizione features e target")
                idx, X, Y = fx.to_XY(ticker, elenco_features, elenco_target, n_timesteps, giorni_previsione, addestramento=True)

                print("Scaler")
                X_train = X.reshape((-1, n_timesteps * n_features))
                Y_train = Y.reshape((-1, giorni_previsione * n_targets))
                X_scaler = clone(scaler)
                Y_scaler = clone(scaler)
                X_train = X_scaler.fit_transform(X_train)
                Y_train = Y_scaler.fit_transform(Y_train)
                X_train = X_train.reshape((-1, n_timesteps, n_features))
                Y_train = Y_train.reshape((-1, giorni_previsione, n_targets))

                X = np.vstack((X, X_train))        
                Y = np.vstack((Y, Y_train))
        except Exception as e:
            print(e)
            continue
    print("Salvataggio X e Y su file")
    np.save('X', X)
    np.save('Y', Y)

Download lista ticker
Download dati ticker validazione
Calcolo indicatori ticker validazione
1 di 1000: Ticker UMBF
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
2 di 1000: Ticker GSBC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler



KeyboardInterrupt



In [ ]:
def rd(hypermodel, X_train, Y_train, X_val, Y_val):
    random_tuner = RandomSearch(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        directory='random_search',
        project_name='random_tuning'
    )
    random_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    random_tuner.search(X_train, Y_train, epochs=epochs, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    # Ottieni il miglior modello e i migliori iperparametri
    best_model_random = random_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_random = random_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Random Search:", best_hyperparameters_random.values)
    
    return best_model_random, best_hyperparameters_random

In [ ]:
def bay(hypermodel, X_train, Y_train, X_val, Y_val):
    bayesian_tuner = BayesianOptimization(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        directory='bayesian_search',
        project_name='bayesian_tuning'
    )
    
    bayesian_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    bayesian_tuner.search(X_train, Y_train, epochs=100, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_bayesian = bayesian_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_bayesian = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Bayesian Optimization:", best_hyperparameters_bayesian.values)
    
    return best_model_bayesian, best_hyperparameters_bayesian


In [ ]:
def hb(hypermodel, X_train, Y_train, X_val, Y_val):
    hyperband_tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=30,
        seed=seed,
        directory='hyperband_search',
        project_name='hyperband_tuning'
    )
    
    hyperband_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    hyperband_tuner.search(X_train, Y_train, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_hyperband = hyperband_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_hyperband = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Hyperband:", best_hyperparameters_hyperband.values)
    
    return best_model_hyperband, best_hyperparameters_hyperband


In [ ]:
#model, par = rd(hypermodel, X, Y, X_val, Y_val)

In [ ]:
model, par = bay(hypermodel, X, Y, X_val, Y_val)